In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.integrate as integrate
import ipyvolume as ipv
import importlib
import sympy as sym

In [ ]:
class LensElem():    
    def __init__(self, data):
        self.curvatureRadius = data[0]/1000
        self.thickness = data[1]/1000
        self.IOR = data[2]
        self.apertureRadius = data[3]/2/1000
    
    def __repr__(self):
        return "Curvature Radius {0}\nThickness {1}\nIOR {2}\nAperture Diameter {3}\n".format(self.curvatureRadius, self.thickness, self.IOR, self.apertureRadius)

In [ ]:
class Ray():
    def __init__(self, o, d):
        self.o = o
        self.d = d/np.linalg.norm(d)

    def march(self, t):
        return self.o + self.d * t

    def sphereIntersect(self, c, r):
        ox = self.o[0]
        oy = self.o[1]
        dx = self.d[0]
        dy = self.d[1]
        cx = c[0]
        cy = c[1]
        t = sym.Symbol('t')
        eq = (ox + t * dx - cx) ** 2 + (oy + t * dy - cy) ** 2 - r ** 2
        return sym.solve(eq, t)
    
    def __repr__(self):
        return "origin {0}\ndirection {1}\n".format(self.o, self.d)

In [ ]:
# wide 22mm camera lens system
data_w22mm = np.array([
# curvature radius, thickness, IOR, aperture diameter
35.98738, 1.21638, 1.54, 23.716,
11.69718, 9.9957, 1.0, 17.996,
13.08714, 5.12622, 1.772, 12.364,
-22.63294, 1.76924, 1.617, 9.812,
71.05802, 0.8184, 1.0, 9.152,
0.0, 2.27766, 0.0, 8.756,
-9.58584, 2.43254, 1.617, 8.184,
-11.28864, 0.11506, 1.0, 9.152,
-166.7765, 3.09606, 1.713, 10.648,
-7.5911, 1.32682, 1.805, 11.44,
-16.7662, 3.98068, 1.0, 12.276,
-7.70286, 1.21638, 1.617, 13.42,
-11.97328, 0.0, 1.0, 17.996,
])
data_w22mm = data_w22mm.reshape((-1,4))
num_lens_elem = data_w22mm.shape[0]

In [ ]:
lens = []
rearZ = 0
frontZ = 0
for i in range(0, num_lens_elem):
    e = LensElem(data_w22mm[i])
    lens.append(e)
    rearZ = e.thickness
    frontZ += rearZ

plt.figure(figsize=(16,4))
plt.xlim(-0.16,0)
plt.ylim(-0.02,0.02)

ox = frontZ
for e in lens:        
    r = e.curvatureRadius

    if r == 0.0:
#         maxTheta = np.arcsin(e.apertureRadius/np.abs(e.curvatureRadius))
#         theta = np.arange(-maxTheta, maxTheta, 0.01)
        x = np.full((2), -ox)
        y = np.array([100, e.apertureRadius])
        plt.plot(x, y)
        y = np.array([-100, -e.apertureRadius])
        plt.plot(x, y)
    else:
        maxTheta = np.arcsin(e.apertureRadius/np.abs(e.curvatureRadius))
        theta = np.arange(-maxTheta, maxTheta, 0.01)
        x = -(r * np.cos(theta) + (ox - e.curvatureRadius))
        y = r * np.sin(theta)
        plt.plot(x, y)

    ox = ox - e.thickness

rOrigin = np.array([-0.11, 0])
rDir = np.array([np.cos(2/180*np.pi), np.sin(2/180*np.pi)])
ray = Ray(rOrigin, rDir)

zIntercept = frontZ
for i in range(0, num_lens_elem-11):
    r = lens[i].curvatureRadius
    c = np.array([-(zIntercept - r), 0])
    ret = ray.sphereIntersect(c, r)
    hit = ray.march(ret[0])
    plt.plot([ray.o[0], hit[0]], [ray.o[1], hit[1]])
    
    rOrigin = np.array([hit[0], hit[1]])
    ray = Ray(rOrigin, rDir)
    zIntercept -= lens[i].thickness
        
    